In [9]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import pickle

import re
from collections import defaultdict 
# from selenium import webdriver
import time
import urllib.request
# from selenium.webdriver.common.action_chains import ActionChains

#settings
from IPython.core.interactiveshell import InteractiveShell  
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_colwidth', -1)
pd.options.display.max_columns = None

In [2]:
listing_link_list = []
for page in range(1,100): 
    url = 'https://www.srx.com.sg/search/sale/hdb?page=' + str(page) 
    #     Just like HTTP request, you ask the driver to open a browser and go to the following link!
    resp = requests.get(url)

    #     create a soup variable that parses the HTML into a BeautifulSoup object
    soup = BeautifulSoup(resp.text, 'html.parser')
    # print(soup.prettify())
    listing_link_string = soup.find_all('a', {'class': 'listingDetailsDivLink', 'href': True})

    for link in listing_link_string:
        listing_link = "https://www.srx.com.sg" + link["href"]
        listing_link_list.append(listing_link)


    print('End of page ' + str(page))

End of page 1
End of page 2
End of page 3
End of page 4
End of page 5
End of page 6
End of page 7
End of page 8
End of page 9
End of page 10
End of page 11
End of page 12
End of page 13
End of page 14
End of page 15
End of page 16
End of page 17
End of page 18
End of page 19
End of page 20
End of page 21
End of page 22
End of page 23
End of page 24
End of page 25
End of page 26
End of page 27
End of page 28
End of page 29
End of page 30
End of page 31
End of page 32
End of page 33
End of page 34
End of page 35
End of page 36
End of page 37
End of page 38
End of page 39
End of page 40
End of page 41
End of page 42
End of page 43
End of page 44
End of page 45
End of page 46
End of page 47
End of page 48
End of page 49
End of page 50
End of page 51
End of page 52
End of page 53
End of page 54
End of page 55
End of page 56
End of page 57
End of page 58
End of page 59
End of page 60
End of page 61
End of page 62
End of page 63
End of page 64
End of page 65
End of page 66
End of page 67
End 

In [4]:
#create a master_df with attributes under 'about this property' and 'facilities'
# as typical facilities that are found in condo are not found in hdb, i retrieve 2 separate lists of facilities
hdb_df = pd.DataFrame()

count=0 #initialise the first row in df
for link in listing_link_list:
    listing_resp = requests.get(link)
    listing_soup = BeautifulSoup(listing_resp.text, 'html.parser')

    about_this_property_section = listing_soup.find_all("div" ,attrs={"class": "row listing-about"})
    facilities_section = listing_soup.find_all("div" ,attrs={"class": "col-6 col-md-4 facilities-row"}) # facilities exists as span tags with attribute name 'listing-about-facility-span' 


    for item in about_this_property_section:
        property_keys = item.find("div",attrs={"class":"col-5"}).text
        property_values = item.find("div", class_=re.compile("^col-7")).text
        
        hdb_df.loc[count, property_keys]=property_values #for each listing, if there's a corresponding value for the key, append the value. if not, it'll skip that key. 
    for facility in facilities_section:
        facilities_keys = facility.find("span",attrs={"class":"listing-about-facility-span"}).text
        if facilities_keys != np.nan: 
            facilities_values = 1
        else: 
            facilities_values = 0

        hdb_df.loc[count,facilities_keys]=facilities_values
    count+=1


In [5]:
len(hdb_df)
hdb_df.head()


1980

,Property Name,Property Type,Asking,PSF,Built Year,Model,Address,HDB Town,Bedrooms,Bathrooms,Area,Land Tenure,Corner Unit,Renovated,Air Conditioning,Water Heater,Cooker Hob/hood,Balcony,Furnish,Original Condition,Hairdryer,City View,Bombshelter,Park/greenery View,Walk-in-wardrobe,Maidsroom,Intercom,Sea View,Colonial Building,Bathtub,Private Garden,Outdoor Patio,Terrace,Private Lift,Meeting Rooms,Roof Terrace,Swimming Pool View,Jacuzzi,Private Pool,Garage
0,Bukit Batok Street 24,HDB Executive,"$560,000",\n$366 psf (Built-up)\n\t\t\t\t\t\t\t\t,1997,Apartment,290B Bukit Batok Street 24 (653290),Bukit Batok,4,2,\n142 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aljunied Crescent,HDB 4 Rooms,"$480,000",\n$485 psf (Built-up)\n\t\t\t\t\t\t\t\t,1980,Model A,112 Aljunied Crescent (380112),Geylang,3,2,\n92 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,1.0,NaN,NaN,1.0,NaN,NaN,Not Furnished,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sengkang West Way,HDB 5 Rooms,"$568,000",\n$467 psf (Built-up)\n\t\t\t\t\t\t\t\t,2016,Improved,469B Sengkang West Way (792469),Sengkang,3,2,\n113 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,1.0,1.0,1.0,1.0,1.0,NaN,Partially Furnished,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Yishun Ring Road,HDB 4 Rooms,"$308,888",\n$342 psf (Built-up)\n\t\t\t\t\t\t\t\t,1986,Standard,257 Yishun Ring Road (760257),Yishun,2,NaN,\n84 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Marine Crescent,HDB 5 Rooms,"$750,000",\n$580 psf (Built-up)\n\t\t\t\t\t\t\t\t,1975,Standard,43 Marine Crescent (440043),Marine Parade,3,2,\n120 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,NaN,1.0,1.0,1.0,1.0,NaN,Partially Furnished,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
with open('../data/raw/hdb_df.pickle', 'wb') as to_write:
    pickle.dump(hdb_df, to_write)


In [10]:

hdb_df_copy = df.copy()
raw_features = list(df.columns.values)
# raw_features
hdb_df_copy['is_duplicated'] = hdb_df_copy.duplicated(raw_features, keep=False)
dup_listing = hdb_df_copy.loc[hdb_df_copy['is_duplicated']==True]
dup_listing
# len(dup_listing)

,Property Name,Property Type,Asking,PSF,Built Year,Model,Address,HDB Town,Bedrooms,Bathrooms,Area,Land Tenure,Corner Unit,Renovated,Air Conditioning,Water Heater,Cooker Hob/hood,Balcony,Furnish,Original Condition,Hairdryer,City View,Bombshelter,Park/greenery View,Walk-in-wardrobe,Maidsroom,Intercom,Sea View,Colonial Building,Bathtub,Private Garden,Outdoor Patio,Terrace,Private Lift,Meeting Rooms,Roof Terrace,Swimming Pool View,Jacuzzi,Private Pool,Garage,is_duplicated
4,Marine Crescent,HDB 5 Rooms,"$750,000",\n$580 psf (Built-up)\n\t\t\t\t\t\t\t\t,1975,Standard,43 Marine Crescent (440043),Marine Parade,3,2,\n120 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,NaN,1.0,1.0,1.0,1.0,NaN,Partially Furnished,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
6,Bedok Reservoir Road,HDB 3 Rooms,"$320,000",\n$437 psf (Built-up)\n\t\t\t\t\t\t\t\t,1981,New Generation,619 Bedok Reservoir Road (470619),Bedok,2,2,\n68 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,NaN,1.0,1.0,1.0,1.0,NaN,Fully Furnished,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
7,Centrale 8 At Tampines,HDB 4 Rooms,"$670,000",\n$750 psf (Built-up)\n\t\t\t\t\t\t\t\t,2014,Model A,519A Tampines Central 8 (521519),Tampines,3,2,\n83 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,1.0,1.0,1.0,1.0,1.0,1.0,Partially Furnished,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
14,Ang Mo Kio Avenue 5,HDB 4 Rooms,"$440,000",\n$413 psf (Built-up)\n\t\t\t\t\t\t\t\t,1980,New Generation,649 Ang Mo Kio Avenue 5 (560649),Ang Mo Kio,3,2,\n99 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,1.0,1.0,1.0,1.0,1.0,NaN,Not Furnished,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
16,Sengkang West Way,HDB 5 Rooms,"$548,000",\n$459 psf (Built-up)\n\t\t\t\t\t\t\t\t,2015,Improved,452A Sengkang West Way (791452),Sengkang,3+1,2,\n111 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,1.0,1.0,1.0,1.0,1.0,NaN,Partially Furnished,NaN,1.0,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,Edgefield Plains,HDB 4 Rooms,"$400,000",\n$404 psf (Built-up)\n\t\t\t\t\t\t\t\t,2012,Model A,615A Edgefield Plains (821615),Punggol,3,2,\n92 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,1.0,1.0,1.0,1.0,1.0,NaN,Partially Furnished,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1965,Upper Serangoon View,HDB 4 Rooms,"$450,000",\n$450 psf (Built-up)\n\t\t\t\t\t\t\t\t,2014,Model A,476B Upper Serangoon View (532476),Hougang,3,2,\n93 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1966,Tampines Street 84,HDB 4 Rooms,"$425,000",\n$380 psf (Built-up)\n\t\t\t\t\t\t\t\t,1988,Model A,871 Tampines Street 84 (520871),Tampines,3,2,\n104 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,NaN,NaN,NaN,1.0,NaN,NaN,Not Furnished,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1970,Trivelis,HDB 3 Rooms,"$570,000",\n$882 psf (Built-up)\n\t\t\t\t\t\t\t\t,2015,Model A,311C Clementi Avenue 4 (123311),Clementi,2,2,\n60 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [25]:
dup_listing.loc[dup_listing['Address']=='43 Marine Crescent (440043)']


,Property Name,Property Type,Asking,PSF,Built Year,Model,Address,HDB Town,Bedrooms,Bathrooms,Area,Land Tenure,Corner Unit,Renovated,Air Conditioning,Water Heater,Cooker Hob/hood,Balcony,Furnish,Original Condition,Hairdryer,City View,Bombshelter,Park/greenery View,Walk-in-wardrobe,Maidsroom,Intercom,Sea View,Colonial Building,Bathtub,Private Garden,Outdoor Patio,Terrace,Private Lift,Meeting Rooms,Roof Terrace,Swimming Pool View,Jacuzzi,Private Pool,Garage,is_duplicated
4,Marine Crescent,HDB 5 Rooms,"$750,000",\n$580 psf (Built-up)\n\t\t\t\t\t\t\t\t,1975,Standard,43 Marine Crescent (440043),Marine Parade,3,2,\n120 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,NaN,1.0,1.0,1.0,1.0,NaN,Partially Furnished,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
203,Marine Crescent,HDB 5 Rooms,"$750,000",\n$580 psf (Built-up)\n\t\t\t\t\t\t\t\t,1975,Standard,43 Marine Crescent (440043),Marine Parade,3,2,\n120 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,NaN,1.0,1.0,1.0,1.0,NaN,Partially Furnished,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [26]:
dup_listing.loc[dup_listing['Address']=='619 Bedok Reservoir Road (470619)']


,Property Name,Property Type,Asking,PSF,Built Year,Model,Address,HDB Town,Bedrooms,Bathrooms,Area,Land Tenure,Corner Unit,Renovated,Air Conditioning,Water Heater,Cooker Hob/hood,Balcony,Furnish,Original Condition,Hairdryer,City View,Bombshelter,Park/greenery View,Walk-in-wardrobe,Maidsroom,Intercom,Sea View,Colonial Building,Bathtub,Private Garden,Outdoor Patio,Terrace,Private Lift,Meeting Rooms,Roof Terrace,Swimming Pool View,Jacuzzi,Private Pool,Garage,is_duplicated
6,Bedok Reservoir Road,HDB 3 Rooms,"$320,000",\n$437 psf (Built-up)\n\t\t\t\t\t\t\t\t,1981,New Generation,619 Bedok Reservoir Road (470619),Bedok,2,2,\n68 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,NaN,1.0,1.0,1.0,1.0,NaN,Fully Furnished,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
207,Bedok Reservoir Road,HDB 3 Rooms,"$320,000",\n$437 psf (Built-up)\n\t\t\t\t\t\t\t\t,1981,New Generation,619 Bedok Reservoir Road (470619),Bedok,2,2,\n68 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,NaN,1.0,1.0,1.0,1.0,NaN,Fully Furnished,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [27]:
dup_listing['Address'].nunique()

342

In [28]:
dup_listing.loc[dup_listing['Address']=='871 Tampines Street 84 (520871)']


,Property Name,Property Type,Asking,PSF,Built Year,Model,Address,HDB Town,Bedrooms,Bathrooms,Area,Land Tenure,Corner Unit,Renovated,Air Conditioning,Water Heater,Cooker Hob/hood,Balcony,Furnish,Original Condition,Hairdryer,City View,Bombshelter,Park/greenery View,Walk-in-wardrobe,Maidsroom,Intercom,Sea View,Colonial Building,Bathtub,Private Garden,Outdoor Patio,Terrace,Private Lift,Meeting Rooms,Roof Terrace,Swimming Pool View,Jacuzzi,Private Pool,Garage,is_duplicated
269,Tampines Street 84,HDB 4 Rooms,"$488,000",\n$436 psf (Built-up)\n\t\t\t\t\t\t\t\t,1888,Model A,871 Tampines Street 84 (520871),Tampines,3,2,\n104 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,NaN,1.0,1.0,1.0,1.0,NaN,Partially Furnished,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
507,Tampines Street 84,HDB 4 Rooms,"$488,000",\n$436 psf (Built-up)\n\t\t\t\t\t\t\t\t,1888,Model A,871 Tampines Street 84 (520871),Tampines,3,2,\n104 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,NaN,1.0,1.0,1.0,1.0,NaN,Partially Furnished,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1003,Tampines Street 84,HDB 4 Rooms,"$425,000",\n$380 psf (Built-up)\n\t\t\t\t\t\t\t\t,1988,Model A,871 Tampines Street 84 (520871),Tampines,3,2,\n104 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,NaN,NaN,NaN,1.0,NaN,NaN,Not Furnished,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1529,Tampines Street 84,HDB 4 Rooms,"$488,000",\n$436 psf (Built-up)\n\t\t\t\t\t\t\t\t,1888,Model A,871 Tampines Street 84 (520871),Tampines,3,2,\n104 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,NaN,1.0,1.0,1.0,1.0,NaN,Partially Furnished,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1740,Tampines Street 84,HDB 4 Rooms,"$488,000",\n$436 psf (Built-up)\n\t\t\t\t\t\t\t\t,1888,Model A,871 Tampines Street 84 (520871),Tampines,3,2,\n104 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,NaN,1.0,1.0,1.0,1.0,NaN,Partially Furnished,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1952,Tampines Street 84,HDB 4 Rooms,"$488,000",\n$436 psf (Built-up)\n\t\t\t\t\t\t\t\t,1888,Model A,871 Tampines Street 84 (520871),Tampines,3,2,\n104 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,NaN,1.0,1.0,1.0,1.0,NaN,Partially Furnished,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1966,Tampines Street 84,HDB 4 Rooms,"$425,000",\n$380 psf (Built-up)\n\t\t\t\t\t\t\t\t,1988,Model A,871 Tampines Street 84 (520871),Tampines,3,2,\n104 sqm (Built-up)\n\t\t\t\t\t\t\t\t,LEASEHOLD/99 years,NaN,NaN,NaN,1.0,NaN,NaN,Not Furnished,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
